In [2]:
import csv
import itertools
import operator
import numpy as np
import nltk
import syl
import sys
from datetime import datetime
from utils import *

import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
nltk.download("book")

[nltk_data] Downloading collection u'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package dependency_treebank is already up-to-date!
[nltk_data]    | Download

True

In [4]:
vocabulary_size = 5000
unknown_token = "UNKNOWN_TOKEN"

#Used to identify what words to start a line with and what words often proceed it.
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

print "Reading Poems..."
with open('data/america.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    #turn poems into lines
    sentences = itertools.chain(*[nltk.sent_tokenize(x[1].decode('utf-8').lower()) for x in reader])
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print "Parsed %d sentences." % (len(sentences))

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

#find the frequency of words in poems
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique word tokens" % len(word_freq.items())

#As RNN uses a vector, connect the vocab and the relative frequency it appears
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % len(index_to_word)
print "The least frequent word in our vocabulary is %s and appeared %d times" % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]


Reading Poems...
Parsed 1809 sentences.
Found 7080 unique word tokens
Using vocabulary size 5000.
The least frequent word in our vocabulary is protest and appeared 1 times

Example sentence: 'SENTENCE_START although she feeds me bread of bitterness,
and sinks into my throat her tiger's tooth,
stealing my breath of life, i will confess
i love this cultured hell that tests my youth! SENTENCE_END'

Example sentence after Pre-processing: '[u'SENTENCE_START', u'although', u'she', u'feeds', u'me', u'bread', u'of', 'UNKNOWN_TOKEN', u',', u'and', u'sinks', u'into', u'my', u'throat', u'her', 'UNKNOWN_TOKEN', u"'s", u'tooth', u',', u'stealing', u'my', u'breath', u'of', u'life', u',', u'i', u'will', u'confess', u'i', u'love', u'this', u'cultured', u'hell', u'that', 'UNKNOWN_TOKEN', u'my', u'youth', u'!', u'SENTENCE_END']'


In [5]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [6]:
# Print an training data example
x_example, y_example = X_train[17], y_train[17]
print "x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example)


x:
SENTENCE_START o beautiful for heroes proved in UNKNOWN_TOKEN UNKNOWN_TOKEN , who more than self their country loved , and mercy more than life !
[2, 119, 370, 13, 415, 1339, 9, 4999, 4999, 0, 57, 64, 135, 641, 24, 109, 1086, 0, 5, 2121, 64, 135, 106, 27]

y:
o beautiful for heroes proved in UNKNOWN_TOKEN UNKNOWN_TOKEN , who more than self their country loved , and mercy more than life ! SENTENCE_END
[119, 370, 13, 415, 1339, 9, 4999, 4999, 0, 57, 64, 135, 641, 24, 109, 1086, 0, 5, 2121, 64, 135, 106, 27, 3]


In [7]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

In [8]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [9]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

RNNNumpy.forward_propagation = forward_propagation

In [10]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNNNumpy.predict = predict

In [11]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print o.shape
print o

(3, 5000)
[[0.00019988 0.00020088 0.00020021 ... 0.00020033 0.00019908 0.00019903]
 [0.0002001  0.00019904 0.0002001  ... 0.00020021 0.00019869 0.00020283]
 [0.00019986 0.00020038 0.000201   ... 0.00019921 0.0002017  0.00019994]]


In [12]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [13]:
# Limit to 1000 examples to save time
print "Expected Loss for random predictions: %f" % np.log(vocabulary_size)
print "Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000])

Expected Loss for random predictions: 8.517193
Actual loss: 8.517318


In [14]:
#Use BPTT Algorithm to calculate gradients for Stochastic Gradient Descent
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt

In [15]:
#Need to check to see if approximate slope at a point is close to partial derviative from bptt
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)

RNNNumpy.gradient_check = gradient_check

# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [16]:
#SGD is used to train the weights
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sdg_step


In [17]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [18]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

100 loops, best of 3: 12.6 ms per loop


In [19]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:500], y_train[:500], nepoch=10, evaluate_loss_after=1)

2018-04-08 14:32:31: Loss after num_examples_seen=0 epoch=0: 8.517339
2018-04-08 14:33:32: Loss after num_examples_seen=500 epoch=1: 6.584341
2018-04-08 14:34:28: Loss after num_examples_seen=1000 epoch=2: 6.204543
2018-04-08 14:35:23: Loss after num_examples_seen=1500 epoch=3: 6.113085
2018-04-08 14:36:25: Loss after num_examples_seen=2000 epoch=4: 6.054230
2018-04-08 14:37:36: Loss after num_examples_seen=2500 epoch=5: 6.001613
2018-04-08 14:39:12: Loss after num_examples_seen=3000 epoch=6: 5.951661
2018-04-08 14:41:12: Loss after num_examples_seen=3500 epoch=7: 5.910547
2018-04-08 14:43:12: Loss after num_examples_seen=4000 epoch=8: 5.866789
2018-04-08 14:45:12: Loss after num_examples_seen=4500 epoch=9: 5.858177


In [20]:
def num_syls(index):

    s=index_to_word[index]
    
    ans = 0
    vowels = "aeiouy"
    vowel_before = False
    for c in s:
        if c in vowels:
            if not vowel_before:
                ans += 1
                vowel_before = True
        else:
            #if vowel_before and c == 'e':
             #   ans -= 1
            vowel_before = False
    '''
    for c in s:
        if c == 'e':
            ans += 1

    if len(s)>=2 and s[-2]=='e' and s[-1]=='d':
        if len(s)>=3 and s[-3] != 't' and s[-3] != 'd':
            ans -= 1

    if len(s)>=3 and s[-3]=='e' and s[-2]=='l' and s[-1]=='y':
        ans -= 1
''' 
    if len(s)>=2 and ans>1 and s[-1] == 'e' and s[-2] not in vowels:
        ans -= 1

    return ans

In [21]:
def generate_sentence(model, syllables):
    # We start the sentence with the start token and initialize the 
    new_sentence = [word_to_index[sentence_start_token]]
    current_number_syllables = 0
    done = False
    
    # Repeat until we get an end token
    while current_number_syllables < syllables:
        next_word_probs, _ = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            x = np.argmax(samples)
        
            if x > len(index_to_word):
                x = len(index_to_word)-1
        
            if ((len(index_to_word[x])>1 or index_to_word[x] in 'IAia') and num_syls(x) + current_number_syllables  <= syllables):  
                sampled_word = x
            
            if num_syls(x) == 0:
                sampled_word = word_to_index[unknown_token]
                
        if not (sampled_word == word_to_index[sentence_start_token] or sampled_word == word_to_index[sentence_end_token]):
            new_sentence.append(sampled_word)
            current_number_syllables += num_syls(x)
        
    sentence_str = [index_to_word[x] for x in new_sentence[1:]]
    return sentence_str

In [27]:
def haiku():
    sent = " "
    sent = generate_sentence(model, 5)
    sentTwo = generate_sentence(model, 7)
    sentThree = generate_sentence(model, 5)
    print " ".join(sent)
    print " ".join(sentTwo)
    print " ".join(sentThree)
    
haiku()

    

he worked so us
charm jack better beneath round
but in the before
